### using shared memory object previously created to run OnACID

In [3]:
import numpy as np
from multiprocessing.shared_memory import SharedMemory
import caiman as cm
from caiman.source_extraction import cnmf
from caiman.source_extraction.cnmf.online_cnmf import demix1p
from caiman.utils.nn_models import (fit_NL_model, create_LN_model, quantile_loss, rate_scheduler)

In [4]:
# Attach to the existing shared memory block
existing_shm = SharedMemory(name="shared-mem")

c = np.ndarray((8410, 512, 512), dtype=np.int16, buffer=existing_shm.buf)

In [6]:
c.nbytes

4409262080

In [7]:
c.data

In [8]:
fname_init = "/home/clewis7/caiman_data/example_movies/demoMovie.tif"

In [9]:
init_batch = 200  # number of frames to use for initialization
T = 2000           # total number of frames 
fr = 15

In [34]:
params_dict = {'fnames': fname_init,
               'fr': fr,
             #  'method_init': 'greedy_roi',
               'K': 10,
               'gSig': (6, 6),
          #     'merge_thr': .85,
               'p': 1,
             #  'tsub': 1,
             #  'ssub': 1,
               'nb': 2,
               'init_batch': init_batch,
               'init_method': 'bare',
               'normalize': True,
               'expected_comps': 500,
               'rval_thr': .8,
               'min_SNR': 1.0,
               'min_num_trial': 10,
               'motion_correct': True,# flag for performing motion correction
               'ds_factor': 1,
               'sniper_mode': True,
               'dist_shape_update': True,
               'epochs': 2,
               'max_shifts_online': 10,
               'pw_rigid': False,
              # 'rf': 24,
              # 'stride': 12,
            #   'gSig_filt': None,        # size of high pass spatial filtering, used in 1p data
             #  'use_cnn': True,
       #        'min_cnn_thr': 0.8,
        #       'cnn_lowest': 0.1,
               'decay_time': 1.0,
               'show_movie': False
              }
opts = cnmf.params.CNMFParams(params_dict=params_dict)

In [35]:
cnm2 = cnmf.online_cnmf.OnACID(dview=None, params=opts)
cnm2.initialize_online(T=T)

In [36]:
from multiprocessing import Process, Queue
from time import time, sleep

In [39]:
q = Queue()
cnm2.t_wait = []
cnm2.t_motion = []
cnm2.t_fit = []
realtime = []

def append_to_queue(q, init_batch, T, fr):
    t_start = time()
    for t in range(init_batch, T):
        # read frame and append to queue
        frame = c[t]
        q.put(frame)
        sleep(max(0, (t+1-init_batch)/fr - time()+t_start))

producer = Process(target=append_to_queue, args=(q,init_batch,T,fr))
producer.start()

# process first frame
t = init_batch
# read form queue (wait for next frame if empty)
t0=time()
frame = q.get().astype(np.float32)
cnm2.t_wait.append(time()-t0)
t_start = time() + (1-init_batch)/fr
# motion correct
t0 = time()
frame = cnm2.mc_next(t, frame)
cnm2.t_motion.append(time()-t0)
# fit
t0 = time()
cnm2.fit_next(t, frame.ravel(order='F'))
cnm2.t_fit.append(time()-t0)
rt = time() <= t_start + t/fr
realtime.append(rt)

# process remaining frames
for t in range(init_batch+1, T):
    # read form queue (wait for next frame if empty)
    t0=time()
    frame = q.get().astype(np.float32)
    cnm2.t_wait.append(time()-t0)
    # motion correct
    t0 = time()
    frame = cnm2.mc_next(t, frame)
    cnm2.t_motion.append(time()-t0)
    # fit
    t0 = time()
    cnm2.fit_next(t, frame.ravel(order='F'))
    cnm2.t_fit.append(time()-t0)
    rt = time() <= t_start + t/fr
    realtime.append(rt)
    # add code to display whatever you want in order to guide the closed-loop experiment below
    print('Realtime: ' + ("\x1b[32mTrue\x1b[0m" if rt else "\x1b[31mFalse\x1b[0m"), end="  \r", flush=True)

2023-02-27 16:26:13.755941: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


2023-02-27 16:26:14.845104: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [ ]:
type(cnm2)

In [ ]:
temporal = list(cnm2.estimates.C)

In [ ]:
len(temporal)

In [ ]:
from fastplotlib import Plot

In [ ]:
plot = Plot()

In [ ]:
plot.add_line_stack(temporal)

In [ ]:
plot.show()

In [ ]:
plot.auto_scale()